## 1. Clone the repository

In [ ]:
!git clone https://github.com/cyrilsofdevpro/SofAi.git
!ls -la
!ls -la SofAi
!ls -la SofAi/backend

## 2. Install dependencies (minimal set for fast startup)

In [ ]:
%cd SofAi/backend
# Install only what we need for a quick demo (SKIP_MODEL_LOAD=1)
!pip install --quiet fastapi uvicorn pyngrok flask-cors

## 3. Start FastAPI backend (with dummy model for speed)

In [ ]:
import os, time, subprocess
import sys

# Ensure we're in the backend directory
os.chdir('/root/SofAi/backend')
print(f"Current directory: {os.getcwd()}")
print(f"Files in current directory: {os.listdir('.')}")

# Set environment variable to use dummy model (fast startup, no HF model load)
os.environ["SKIP_MODEL_LOAD"] = "1"

# Kill any existing uvicorn process
os.system("pkill -f 'uvicorn' 2>/dev/null || true")
time.sleep(1)

# Start uvicorn in background: bind to 0.0.0.0 so ngrok can reach it
# Use full path to ensure Python finds the module
print("Starting uvicorn backend...")
backend_dir = '/root/SofAi/backend'
get_ipython().system_raw(
    f'cd {backend_dir} && python -m uvicorn main:app --host 0.0.0.0 --port 8000 --log-level info > uvicorn.log 2>&1 &'
)

# Wait for server to start and show startup logs
time.sleep(3)
print("\n=== Uvicorn Startup Logs ===")
!tail -n 300 uvicorn.log

## 4. Verify backend is listening locally

In [ ]:
import time
time.sleep(1)

print("Checking if backend is listening...")
!curl -sS http://127.0.0.1:8000/health && echo "\n✓ /health endpoint working" || echo "✗ /health failed"

print("\nTesting /chat endpoint...")
!curl -sS -X POST http://127.0.0.1:8000/chat \
  -H "Content-Type: application/json" \
  -d '{"message":"hello from colab","session_id":"test123"}' && echo "\n✓ /chat endpoint working" || echo "✗ /chat failed"

## 5. Start ngrok tunnel (public URL)

In [ ]:
from pyngrok import ngrok
from getpass import getpass

# Option A: Use getpass to securely enter token (recommended)
print("Enter your ngrok authtoken (from https://dashboard.ngrok.com/auth/your-authtoken):")
token = getpass("ngrok authtoken (will not echo): ")

if token.strip():
    ngrok.set_auth_token(token.strip())
else:
    print("⚠️ No token entered. Using default (may have connection limits).")

# Create tunnel: port 8000, TLS enabled for security
print("Creating ngrok tunnel...")
tunnel = ngrok.connect(8000, bind_tls=True)
PUBLIC_URL = tunnel.public_url

print(f"\n✓ Public ngrok URL: {PUBLIC_URL}")
print(f"\n  Use this URL to access your backend from anywhere:")
print(f"    Health check: {PUBLIC_URL}/health")
print(f"    Chat endpoint: {PUBLIC_URL}/chat (POST)")

## 6. Test public endpoint via ngrok

In [ ]:
import subprocess

# Extract the public URL from the tunnel object
from pyngrok import ngrok
tunnels = ngrok.get_tunnels()
public_url = tunnels[0].public_url if tunnels else None

if public_url:
    print(f"Testing public endpoint: {public_url}")
    print("\n=== Public /health ===")
    !curl -sS {public_url}/health && echo "\n✓ Public health check passed" || echo "✗ Public health check failed"
    
    print("\n=== Public /chat POST ===")
    !curl -sS -X POST {public_url}/chat \
      -H "Content-Type: application/json" \
      -d '{"message":"test from public url","session_id":"public_test"}' && echo "\n✓ Public chat endpoint passed" || echo "✗ Public chat endpoint failed"
else:
    print("No public URL found. Make sure ngrok tunnel is active.")

## 7. (Optional) Check running processes and tunnel status

In [ ]:
print("=== Running Processes ===")
!ps aux | grep -E "uvicorn|python" | grep -v grep

print("\n=== ngrok Tunnel Status ===")
!curl -s http://127.0.0.1:4040/api/tunnels | python -m json.tool

## 8. Frontend Setup (if using local React frontend)

If you want to test the React UI locally with this Colab backend:

1. In your local terminal, from `SofAI/frontend`:
   ```bash
   export VITE_API_BASE=https://<your-ngrok-url>  # use the PUBLIC_URL from step 5
   npm run dev
   ```

2. Open http://localhost:3001 in your browser and send messages.

The frontend will call your public ngrok endpoint. CORS is enabled in the FastAPI backend, so cross-origin requests should work.

## 9. Debugging: View backend logs if something fails

In [ ]:
# Show latest uvicorn logs (useful if endpoint tests failed)
print("=== Latest Uvicorn Logs ===")
!tail -n 500 uvicorn.log

## 10. Stop backend (when done)

Run this cell to cleanly stop uvicorn and ngrok tunnels.

In [ ]:
import os
from pyngrok import ngrok

print("Stopping uvicorn...")
os.system("pkill -f 'uvicorn' 2>/dev/null || true")

print("Closing ngrok tunnels...")
ngrok.kill()

print("✓ Backend and ngrok stopped.")